In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
from ggplot import *
%matplotlib inline

# import data

In [ ]:
attendance_df = pd.read_csv("../input/nba_2017_attendance.csv");attendance_df.head()

In [ ]:
endorsement_df = pd.read_csv("../input/nba_2017_endorsements.csv");endorsement_df.head()

In [ ]:
valuations_df = pd.read_csv("../input/nba_2017_team_valuations.csv");valuations_df.head()

In [ ]:
salary_df = pd.read_csv("../input/nba_2017_salary.csv");salary_df.head()

In [ ]:
pie_df = pd.read_csv("../input/nba_2017_pie.csv");pie_df.head()

In [ ]:
plus_minus_df = pd.read_csv("../input/nba_2017_real_plus_minus.csv");plus_minus_df.head()

In [ ]:
br_stats_df = pd.read_csv("../input/nba_2017_br.csv");br_stats_df.head()

In [ ]:
elo_df = pd.read_csv("../input/nba_2017_elo.csv");elo_df.head()

In [ ]:
attendance_valuation_df = attendance_df.merge(valuations_df, how="inner", on="TEAM")
attendance_valuation_elo_df = attendance_valuation_df.merge(elo_df, how="inner", on="TEAM")
attendance_valuation_elo_df.head()

## K Means Cluster for palyer

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
#standardize the data first
sc = StandardScaler()
data = sc.fit_transform(br_stats_df.iloc[:,5:].dropna())

#fit into k-means cluster
kmeans = KMeans(n_clusters=7)
kmeans.fit(data)

player_kmeans4 = br_stats_df.copy().dropna()
player_kmeans4["cluster"] = kmeans.labels_
player_kmeans4.head()

In [ ]:
plt.scatter(player_kmeans4.iloc[:,-2], player_kmeans4.iloc[:,-3], c=kmeans.labels_,  s=50, cmap='viridis')

In [ ]:
g = sns.FacetGrid(player_kmeans4, col="cluster")
g = g.map(plt.scatter, "PS/G", "3P%", marker="o")

In [ ]:
p = ggplot(player_kmeans4,aes(x="PS/G", y="2P", color="Pos")) + geom_point(size=200) +facet_grid("cluster")
p + xlab("POINTS/GAME") + ylab("2P") + ggtitle("NBA Players 2016-2017")

## relationship with salary

In [ ]:
player_salary = br_stats_df.merge(salary_df,how = "left",left_on="Player",right_on = "NAME")
player_salary.head(3)

In [ ]:
plt.figure(figsize =(8,10))
sns.boxplot(y="TEAM", x="SALARY", data = player_salary)

Cleveland Cavaliers and Golden State Warriors bhave players with higher salary

In [ ]:
player_salary.loc[player_salary["POSITION"]==' PG',"POSITION"] = 'PG'
player_salary.loc[player_salary["POSITION"]==' SF',"POSITION"] = 'SF'

plt.figure(figsize =(8,10))
sns.boxplot(y="POSITION", x="SALARY", data = player_salary)

[](http://)WOW. The salary of SF is significantly higher than other position. 

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(player_salary.corr(),cmap='coolwarm',annot=True)

Points per game has the stronggest relationship with salary, while FT, FTA, 2P, 2PA, FG, FGA have strong relationship with points per game. Why 2P% is not related? try to standardize the data. No much difference.

In [ ]:
sc = StandardScaler()
data = sc.fit_transform(player_salary.dropna().select_dtypes(include=['float64','int64']))
data = pd.DataFrame(data,index = player_salary.dropna().index, 
                    columns = player_salary.select_dtypes(include=['float64','int64']).columns)
plt.figure(figsize=(15,10))
sns.heatmap(data.corr(),cmap='coolwarm',annot=True)

In [ ]:
sns.lmplot(x="SALARY", y="PS/G", data=player_salary)

Players gain more points tend to have higher salary 

In [ ]:
player_salary[['Player','2P','2PA','2P%','3P','3PA','3P%']].sort_values(
    by=['3P'],ascending = False).drop_duplicates().head(5)

In [ ]:
sns.lmplot(x="2P%", y="3P%", data=player_salary)
plt.ylim(0,0.6)
plt.xlim(0,0.8)

Are those who have high 3 points hit ratio also good at 2 points? Seems not. 

In [ ]:
import statsmodels.api as sm

In [ ]:
train_x = attendance_valuation_elo_df.drop(["ELO","TEAM","CONF"],axis=1)
train_y = attendance_valuation_elo_df["ELO"]

x = sm.add_constant(train_x, has_constant='add')
est = sm.OLS(train_y, x)
est = est.fit()
print(est.summary()) 

In [ ]:
#predict
pred_y= est.predict(x)
pred_y